In [2]:
# In /2_notebooks/03_computer_vision_model.ipynb
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib

# --- 2. Preprocess the Data ---
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
data_dir = pathlib.Path('../data/raw/images')

print(f"Loading images manually from: {data_dir}")

ds_train = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

ds_test = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

CLASS_NAMES = ds_train.class_names
NUM_CLASSES = len(CLASS_NAMES)
print(f"Found {NUM_CLASSES} classes. First 5: {CLASS_NAMES[:5]}")

def format_image(image, label):
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

ds_train = ds_train.map(format_image).prefetch(1)
ds_test = ds_test.map(format_image).prefetch(1)

print(f"Dataset loaded. Class names: {CLASS_NAMES[:5]}...") # Show 5 classes

# --- 3. Create the Model (Transfer Learning) ---
print("Building model with MobileNetV2 base...")
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False, # Don't include the final 1000-class layer
    weights='imagenet'
)
base_model.trainable = False # Freeze the base model

# Add our new "head"
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x) # Our 101-class layer

cv_model = tf.keras.Model(inputs, outputs)

cv_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cv_model.summary()

# --- 4. Train the Model ---
print("Training the model head...")
history = cv_model.fit(ds_train, epochs=5, validation_data=ds_test)

# --- 5. Save the CV Model and Class Names ---
print("Saving CV model...")
cv_model.save('../ml_model/models/dish_classifier_v1.h5')

# Save the class names in a text file
with open('../ml_model/models/food_101_class_names.txt', 'w') as f:
    for class_name in CLASS_NAMES:
        f.write(f"{class_name}\n")

print("CV model and class names saved.")

Loading images manually from: ..\data\raw\images
Found 101000 files belonging to 101 classes.
Using 80800 files for training.
Found 101000 files belonging to 101 classes.
Using 20200 files for validation.
Found 101 classes. First 5: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare']
Dataset loaded. Class names: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare']...
Building model with MobileNetV2 base...


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 101)            │       129,381 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,387,365 (9.11 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Training the model head...
Epoch 1/5
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1817s 718ms/step - accuracy: 0.4823 - loss: 2.0818 - val_accuracy: 0.5650 - val_loss: 1.7178
Epoch 2/5
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1718s 680ms/step - accuracy: 0.5762 - loss: 1.6507 - val_accuracy: 0.5771 - val_loss: 1.6805
Epoch 3/5
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1726s 683ms/step - accuracy: 0.5987 - loss: 1.5521 - val_accuracy: 0.5805 - val_loss: 1.6908
Epoch 4/5
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1707s 676ms/step - accuracy: 0.6091 - loss: 1.4988 - val_accuracy: 0.5782 - val_loss: 1.7032
Epoch 5/5
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1721s 681ms/step - accuracy: 0.6171 - loss: 1.4648 - val_accuracy: 0.5773 - val_loss: 1.7290


Saving CV model...
CV model and class names saved.
